In [1]:
import numpy as np, os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


# Data Load

In [2]:
import random    
seed_num = 48
random.seed(seed_num)

X = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(X)))
random.shuffle(idx)


i = round(X.shape[0]*0.8)
X_train, y_train = X[:i,:,:], y[:i]
X_test, y_test = X[i:,:,:], y[i:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

# get_model, BinaryCrossEntropy function

In [3]:
def get_model():
    seed_num = 48
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(X.shape[1],X.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate = 0.001)
    lstm.compile(optimizer=optimizer, loss = "binary_crossentropy", metrics=['acc'])
    return lstm

In [4]:
def BinaryCrossEntropy(y_true, y_pred):
    m = y_true.shape[1]
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Calculating loss
    loss = -1/m * (np.dot(y_true.T, np.log(y_pred)) + np.dot((1 - y_true).T, np.log(1 - y_pred)))

    return loss

# Model Save

In [5]:
MODEL_SAVE_FOLDER_PATH = './models/'
filepath = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_acc:.4f}.hdf5'
ckpt = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, save_weights_only=False, save_freq='epoch')

model = get_model()
early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop, ckpt], shuffle=False)

Epoch 1/500
37/37 [==============================] - 6s 78ms/step - loss: 0.6255 - acc: 0.6872 - val_loss: 0.7495 - val_acc: 0.4955
Epoch 2/500
37/37 [==============================] - 2s 64ms/step - loss: 0.6240 - acc: 0.6887 - val_loss: 0.7583 - val_acc: 0.4955
Epoch 3/500
37/37 [==============================] - 2s 63ms/step - loss: 0.6265 - acc: 0.6885 - val_loss: 0.7581 - val_acc: 0.4955
Epoch 4/500
37/37 [==============================] - 2s 62ms/step - loss: 0.5813 - acc: 0.6943 - val_loss: 0.6175 - val_acc: 0.6953
Epoch 5/500
37/37 [==============================] - 2s 63ms/step - loss: 0.4594 - acc: 0.7871 - val_loss: 0.5644 - val_acc: 0.7277
Epoch 6/500
37/37 [==============================] - 2s 62ms/step - loss: 0.3944 - acc: 0.8281 - val_loss: 0.5836 - val_acc: 0.7212
Epoch 7/500
37/37 [==============================] - 2s 63ms/step - loss: 0.3458 - acc: 0.8598 - val_loss: 0.6110 - val_acc: 0.7342
Epoch 8/500
37/37 [==============================] - 2s 61ms/step - loss: 0.

In [6]:
#저장된 모델 불러오기
model = tf.keras.models.load_model('/project/guri/Restart/models/14-0.7367.hdf5')

# Base BCE

In [8]:
pred = model.predict(X_test)
# bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
base_bce = BinaryCrossEntropy(pred, y_test)[0]
base_bce

8008.864772323024

# COLS Load, reverse function

In [11]:
# 컬럼이름 로드
COLS = list(pd.read_csv('/project/guri/data/data2d_forposter.csv').columns)

In [13]:
#reverse 함수 생성
def reverse(li):
    return np.logical_not(li).astype(np.float64)

# ALL

In [14]:
results = []
results.append({'feature':'BASELINE', 'bce':base_bce})

for i in tqdm(range(len(COLS))):
    #원본 저장해놓고 for문 하나 끝날 때 다시 복원해준다. - save_col
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,:,i] = reverse(X_test[p,:,i])
        
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})
    X_test[:,:,i] = save_col

  0%|          | 0/4069 [00:00<?, ?it/s]

In [15]:
df1 = pd.DataFrame(results)
df1 = df1.sort_values('bce', ascending=False)
df1.to_csv('./reverse_FI_with_saved_model.csv', index=False)

In [17]:
df1

,feature,bce
2197,409606211,10634.860764
3285,51479005520,9598.850359
1082,67434504,9274.589349
1277,74606211,9222.603923
3588,59011010325,9214.718024
...,...,...
2155,409198530,7929.591817
2191,409490234,7914.460668
306,224275,7910.113561
3458,55390000401,7906.018468


# ALL (7727) - D-1~5
- `X_test[p,:,i]` 부분을 `X_test[p,-5:,i]`로 수정

In [18]:
results = []
results.append({'feature':'BASELINE', 'bce':base_bce})

for i in tqdm(range(len(COLS))):
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,-5:,i] = reverse(X_test[p,-5:,i])
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})
    X_test[:,:,i] = save_col

  0%|          | 0/4069 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df2 = pd.DataFrame(results)
df2 = df2.sort_values('bce', ascending=False)
df2.to_csv('./reverse_FI_d1d5_with_saved_model.csv', index=False)

In [ ]:
df2

# ALL (7727) - D-6~10
- `X_test[p,:,i]` 부분을 `X_test[p,:5,i]`로 수정

In [ ]:
results = []
results.append({'feature':'BASELINE', 'bce':base_bce})

for i in tqdm(range(len(COLS))):
    save_col = X_test[:,:,i].copy()
    for p in range(X_test.shape[0]):
        X_test[p,:5,i] = reverse(X_test[p,:5,i])
    pred = model.predict(X_test)
    bce = BinaryCrossEntropy(pred, y_test)[0]
    results.append({'feature':COLS[i], 'bce':bce})
    X_test[:,:,i] = save_col

In [ ]:
df3 = pd.DataFrame(results)
df3 = df3.sort_values('bce', ascending=False)
df3.to_csv('./reverse_FI_d6d10_with_saved_model.csv', index=False)